In [1]:
import numpy as np 
import pandas as pd 

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

Using TensorFlow backend.


In [2]:
#Vader
import numpy as np
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer as sia 
df = pd.read_csv('Restaurant_Reviews.csv', delimiter = '\t', quoting = 3)
positivity=[]
negativity=[]
overall_sentiment=[]

sid_obj=sia()

for review in df["Review"]:
    sentiment_dict = sid_obj.polarity_scores(review)
    positivity.append(sentiment_dict['pos'])
    negativity.append(sentiment_dict['neg'])
    overall_sentiment.append('pos' if sentiment_dict['pos']> sentiment_dict['neg'] else 'neg' if sentiment_dict['pos']<sentiment_dict['neg'] else 'pos')

df.insert(2,'Pos',positivity)
df.insert(3,'Neg',negativity)
df.insert(4,'Senti',overall_sentiment)

In [3]:
df['Review']= [x.lower() for x in df['Review']]
df['Review'] = df['Review'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

In [4]:
for idx,row in df.iterrows():
    row[0] = row[0].replace('rt',' ')

In [5]:
pd.set_option('display.max_colwidth',-1)

In [6]:
df['Review']= [x.encode('ascii') for x in df['Review']]

In [7]:
df

,Review,Liked,Pos,Neg,Senti
0,b'wow loved this place',1,0.794,0.000,pos
1,b'crust is not good',0,0.000,0.445,neg
2,b'not tasty and the texture was just nasty',0,0.000,0.340,neg
3,b'stopped by during the late may bank holiday off rick steve recommendation and loved it',1,0.322,0.093,pos
4,b'the selection on the menu was great and so were the prices',1,0.272,0.000,pos
...,...,...,...,...,...
995,b'i think food should have flavor and texture and both were lacking',0,0.000,0.000,pos
996,b'appetite instantly gone',0,0.000,0.000,pos
997,b'overall i was not impressed and would not go back',0,0.000,0.221,neg
998,b'the whole experience was underwhelming and i think well just go to ninja sushi next time',0,0.000,0.000,pos


In [8]:
tokenizer = Tokenizer(nb_words=2000, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                   lower=True,split=' ')
df = df.astype({"Review":str})
tokenizer.fit_on_texts(df['Review'].values)

D:\Anaconda\lib\site-packages\keras_preprocessing\text.py:178: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [9]:
X = tokenizer.texts_to_sequences(df['Review'].values)
X = pad_sequences(X)

In [10]:

embed_dim = 128
lstm_out = 196
batch_size = 32

model = Sequential()
model.add(Embedding(2000, embed_dim,input_length = X.shape[1], dropout = 0.2))
model.add(LSTM(lstm_out, dropout_U = 0.2, dropout_W = 0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(196, dropout=0.2, recurrent_dropout=0.2)`
  import sys


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 32, 128)           256000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
Y = pd.get_dummies(df['Senti']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(800, 32) (800, 2)
(200, 32) (200, 2)


In [14]:
model.fit(X_train, Y_train, batch_size = batch_size, nb_epoch = 10, verbose =2 )

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Epoch 1/10
 - 2s - loss: 0.0053 - accuracy: 0.9987
Epoch 2/10
 - 2s - loss: 0.0109 - accuracy: 0.9962
Epoch 3/10
 - 3s - loss: 0.0069 - accuracy: 0.9987
Epoch 4/10
 - 3s - loss: 0.0137 - accuracy: 0.9962
Epoch 5/10
 - 3s - loss: 0.0049 - accuracy: 1.0000
Epoch 6/10
 - 2s - loss: 0.0055 - accuracy: 0.9975
Epoch 7/10
 - 2s - loss: 0.0039 - accuracy: 0.9975
Epoch 8/10
 - 2s - loss: 0.0050 - accuracy: 0.9975
Epoch 9/10
 - 2s - loss: 0.0030 - accuracy: 0.9987
Epoch 10/10
 - 2s - loss: 0.0037 - accuracy: 0.9975


In [15]:
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.2f" % (acc))

Score: 1.46
Validation Accuracy: 0.73
